In [ ]:
import pyarrow.parquet as pq
import pandas as pd
from pymongo import MongoClient

In [ ]:
table = pq.read_table('sdsc_data/output.parquet')
df = table.to_pandas()

In [ ]:
df.head()

In [ ]:
client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['sdsc-data']
#collection.delete_many({}) # Clears out collection

In [ ]:
data = df.to_dict(orient = 'records') # Converts DataFrame to list of dictionaries (JSON-like)

In [ ]:
## START OF POPULATING DATA INTO MONGODB

In [ ]:
from bson import BSON
from datetime import datetime
 
for doc in data: # Have to preprocess some column to fit datatype for MongoDB
    publish_date = datetime.combine(doc['publishdate'], datetime.min.time())
    collection_date = datetime.combine(doc['collectiondate'], datetime.min.time())
    
    publish_date_timestamp = int(publish_date.timestamp() * 1000)
    collection_date_timestamp = int(collection_date.timestamp() * 1000)
    
    doc['publishdate'] = publish_date_timestamp
    doc['collectiondate'] = collection_date_timestamp
    
    doc['author'] = doc['author'].tolist()
    doc['keywords'] = doc['keywords'].tolist()

In [ ]:
collection.insert_many(data)

In [ ]:
## END OF POPULATING DATA INTO MONGODB